## Imports

In [149]:
import numpy as np
from scipy.sparse.linalg import spsolve
from meshplot import plot
import triangle  # MAKE SURE THIS LIBRARY IS INSTALLED
import igl
import math

## Diffusion Curves

In [216]:
class Line:
    # p1 and p2 are tuples representing the start and end points
    # c1 and c2 are tuples representing the colors on each side
    def __init__(self, p1, p2, c1, c2):
        self.p1 = np.array(p1)
        self.p2 = np.array(p2)
        self.c1 = np.array(c1)
        self.c2 = np.array(c2)
    
    # Get n samples points on the line
    def sample_points(self, n=100, offset=0.01):
        points = []
        diff = self.p2 - self.p1
        offset_direction = np.cross(diff, np.array((0, 0, -1.0)))  # Get perpendicular vector
        offset_direction /= np.linalg.norm(offset_direction)
        for i in range(n):
            points.append(self.p1 + diff * i / (n-1) + offset * offset_direction)
        return np.array(points)


class Circle:
    # p are tuples representing the center of the circle
    # r is the radius
    # c1 and c2 are tuples representing the colors on each side
    def __init__(self, p, r, c1, c2):
        self.p = np.array(p)
        self.r = r
        self.c1 = np.array(c1)
        self.c2 = np.array(c2)
    
    # Get n samples on the circle
    def sample_points(self, n=100, offset=0.01):
        points = []
        for i in range(n):
            theta = 2 * np.pi * i / (n-1)
            points.append(self.p + (self.r+offset) * np.array((math.cos(theta), math.sin(theta), 0.0)))
        return np.array(points)


class Spline:
    # NOTE: The spline code is based off code I previously wrote for
    #       the first assignment of CSC 473 (Computer Animation)
    # points is a list of tuples that are the control points
    # tangents is a list of tuples containing the corresponding tangents of the control points
    # c1 and c2 are tuples representing the colors on each side
    def __init__(self, points, tangents, c1, c2):
        assert(len(points) == len(tangents))
        self.points = np.array(points)
        self.tangents = np.array(tangents)
        self.c1 = np.array(c1)
        self.c2 = np.array(c2)
    
    # Get (n * #points) samples points on the line
    def sample_points(self, n=100, offset=0.01):
        n = (len(self.points) - 1) * n
        original_points = []
        offsetted_points = []
        distance_between = 1.0 / (n-1.0)  # Determine amount to jump between the n points
        
        # Calculcate spline curve
        for i in range(n):
            t = i * distance_between
            p = self.evaluate_curve(t)
            original_points.append(p)
        
        # Calculcate tangents to find perpendicular vectors for offsetting
        for i in range(n):
            tangent = None
            if i == 0 and i+1 < n:
                tangent = original_points[i+1] - original_points[i]
            elif i == n-1 and i > 0:
                tangent = original_points[i] - original_points[i-1]
            elif i > 0 and i+1 < n:
                tangent = original_points[i+1] - original_points[i-1]
            else:
                assert(False)
            offset_direction = np.cross(tangent, np.array((0, 0, -1.0)))
            offset_direction /= np.linalg.norm(offset_direction)
            offsetted_points.append(original_points[i] + offset * offset_direction)
            
        return np.array(offsetted_points)
    
    # Evaluate at spline at point t, between 0.0 and 1.0
    def evaluate_curve(self, t):
        n = len(self.points)
        distance_between = 1.0 / (n-1)
        i = int(t / distance_between)  # Determine which segment to interpolate
        if i >= n - 1:
            return self.points[-1]
        t_i = distance_between * i  # T value at which segment starts
        
        # Calculcate hermite spline interpolation
        y0 = self.points[i]
        y1 = self.points[i+1]
        s0 = self.tangents[i]
        s1 = self.tangents[i+1]
        t = (t - t_i) / distance_between
        p = y0 * (2.0 * t**3.0 - 3 * t**2.0 + 1.0)
        p += y1 * (-2.0 * t**3.0 + 3.0 * t**2.0)
        p += s0 * (t**3.0 - 2.0 * t**2.0 + t)
        p += s1 * (t**3.0 - t**2.0)
        return p


GRID_SIZE = 100  # Create NxN grid
BACKGROUND_COLOR = np.array([0.0, 0.0, 0.0])  # Border color
CURVE_OFFSET = 0.005  # For splitting curves for each color

curves = [
    # Border (p1, p2, color1, color2)
    Line((0.0, 0.0, 0.0), (1.0, 0.0, 0.0), BACKGROUND_COLOR, BACKGROUND_COLOR),
    Line((1.0, 0.0, 0.0), (1.0, 1.0, 0.0), BACKGROUND_COLOR, BACKGROUND_COLOR),
    Line((1.0, 1.0, 0.0), (0.0, 1.0, 0.0), BACKGROUND_COLOR, BACKGROUND_COLOR),
    Line((0.0, 1.0, 0.0), (0.0, 0.0, 0.0), BACKGROUND_COLOR, BACKGROUND_COLOR),
    
    # Lines (p1, p2, color1, color2)
    Line((0.25, 0.25, 0.0), (0.25, 0.75, 0.0), (1.0, 0.0, 0.0), (1.0, 0.5, 0.0)),
    Line((0.75, 0.25, 0.0), (0.75, 0.75, 0.0), (0.0, 1.0, 0.0), (0.0, 0.0, 1.0)),
    #Line((0.25, 0.25, 0.0), (0.75, 0.75, 0.0), (0.0, 1.0, 0.0), (0.0, 1.0, 0.0)),
    #Line((0.25, 0.75, 0.0), (0.75, 0.25, 0.0), (1.0, 0.0, 1.0), (1.0, 0.0, 1.0)),
    
    # Circles (center, radius, color1, color2)
    Circle((0.5, 0.5, 0.0), 0.1, (1.0, 0.0, 0.0), (1.0, 1.0, 0.0)),
    
    # Splines ((points, ...), (tangents, ...), color1, color2)
    Spline(((0.1, 0.1, 0.0), (0.1, 0.9, 0.0), (0.9, 0.9, 0.0)),
          ((0.0, 1.0, 0.0), (0.5, 0.5, 0.0), (1.0, 0.0, 0.0)),
           (1.0, 1.0, 1.0), (1.0, 0.0, 0.0))
]

v_b = []  # Constrained vertices
bc_red = []  # Red values
bc_green = []  # Green values
bc_blue = []  # Blue values
vertex_i = 0
segment_i = 0

v_curves = []  # Vertices of curves
v_grid = []  # Vertices of the grid

segments = []  # For drawing curves on-top for visualization
v_segments = []  # Vertices for the segments

# Generate curves
for c in curves:
    # Generate each colored side
    for offset in [-CURVE_OFFSET, CURVE_OFFSET]:
        points = c.sample_points(100, offset)  # Sample curve
        for i in range(len(points)):
            v_b.append(vertex_i)  # Append constrained vertex
            vertex_i += 1
            
            # Add constrained colors
            color = c.c1
            if offset > 0:
                color = c.c2
            bc_red.append(color[0])
            bc_green.append(color[1])
            bc_blue.append(color[2])
        
        # Add to mesh vertices
        v_curves.extend(points)
    
    # Generate line itself for visualization
    segment_points = c.sample_points(100, 0.0)
    for i in range(len(points) - 1):
        segments.append((segment_i, segment_i + 1))
        segment_i += 1
    segment_i += 1
    v_segments.extend(segment_points)

# Generate grid
for x in range(GRID_SIZE+1):
    for y in range(GRID_SIZE+1):
        v_grid.append((x / GRID_SIZE, y / GRID_SIZE, 0.0))
        vertex_i += 1

# Remove duplicate vertices (that are parts of curves) from grid
v_grid = np.array(list(set(tuple(map(tuple, v_grid))) - set(tuple(map(tuple, v_curves)))))

# Convert everything to np arrays
v_curves = np.array(v_curves)
v_segments = np.array(v_segments)
segments = np.array(segments)
v_b = np.array(v_b)
bc_red = np.array(bc_red)
bc_green = np.array(bc_green)
bc_blue = np.array(bc_blue)

# Concat curve and grid vertices
v = np.append(v_curves, v_grid, axis=0)

# Triangulate
triangulation = triangle.triangulate(dict(vertices=v[:,:2], segments=segments))
f = triangulation["triangles"]

# Laplace equation (https://libigl.github.io/libigl-python-bindings/tut-chapter2/#laplace-equation)
v_all = np.arange(v.shape[0])
v_in = np.setdiff1d(v_all, v_b)

l = igl.cotmatrix(v, f)
l_ii = l[v_in, :]
l_ii = l_ii[:, v_in]

l_ib = l[v_in, :]
l_ib = l_ib[:, v_b]

# Solve for each color channel
red_in = spsolve(-l_ii, l_ib.dot(bc_red))
green_in = spsolve(-l_ii, l_ib.dot(bc_green))
blue_in = spsolve(-l_ii, l_ib.dot(bc_blue))

# Set face colors
c = np.zeros(v.shape)
# Constrained colors
c[v_b, 0] = bc_red
c[v_b, 1] = bc_green
c[v_b, 2] = bc_blue
# Solved colors
c[v_in, 0] = red_in
c[v_in, 1] = green_in
c[v_in, 2] = blue_in

# Plot
p = plot(v, f, c, shading={"wireframe": False})
p.add_edges(v_segments, segments, shading={"line_width": 1.0, "line_color": "black"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4999999…

1